Import libraries for data cleaning 

In [1]:


from notebooks.db_connection import get_db_engine
from notebooks.db_connection import execute_query
import pandas as pd

Connection Successful!


Reading CSV file

In [2]:
df_themes = pd.read_csv(r"C:\Users\aless\Desktop\Uni\Dataset\themes.csv\themes.csv")

Top records of CSV file: themes.csv

In [3]:
df_themes

,id,theme
0,1000001,Humanity and the world around us
1,1000001,Crude humor and satire
2,1000001,Moving relationship stories
3,1000001,Emotional and captivating fantasy storytelling
4,1000001,Surreal and thought-provoking visions of life ...
...,...,...
125636,1835643,Noir and dark crime dramas
125637,1835643,Intriguing and suspenseful murder mysteries
125638,1849827,Faith and religion
125639,1849827,Faith and spiritual journeys


Checking for total dimension of the csv

In [4]:
df_themes.shape

(125641, 2)

Rename 'id' column in 'id_theme'. "inplace = True" modifies directly the df without creating a copy

In [5]:
df_themes.rename(columns = {"id":"id_movie"} , inplace = True)


In [6]:
df_themes.columns

Index(['id_movie', 'theme'], dtype='object')

Check info about column type

In [7]:
df_themes.dtypes

id_movie     int64
theme       object
dtype: object

Correction of data type of the records

In [8]:
df_themes['id_movie'] = df_themes['id_movie'].astype('Int64')
df_themes['theme'] = df_themes['theme'].fillna(pd.NA).astype('string')

df_themes.dtypes

id_movie             Int64
theme       string[python]
dtype: object

Check if there are duplicates

In [9]:
df_themes.duplicated().sum()


np.int64(0)

In [10]:
df_themes.drop_duplicates(inplace = True)
df_themes.duplicated().sum()

np.int64(0)

Check the missing values

In [11]:
df_themes.isnull().sum()

id_movie    0
theme       0
dtype: int64

Dropping records with missing value 

In [12]:
df_themes = df_themes.dropna()
df_themes.isnull().sum()

id_movie    0
theme       0
dtype: int64

Remove themes where id_movie doesn't exist

In [13]:
valid_movies = pd.read_csv("movies_valid_ids.csv")
df_themes = df_themes[df_themes["id_movie"].isin(valid_movies["id_movie"])]
df_themes.shape

(125641, 2)

Connecting to PostgreSQL database and creating the table

In [14]:
engine = get_db_engine()
sql = \
    """CREATE TABLE IF NOT EXISTS themes (
            id BIGSERIAL PRIMARY KEY,
            id_movie INTEGER CHECK (id_movie>=0),
            theme TEXT CHECK (theme<>'')
            );
    """
execute_query(sql)

Query Executed Successfully!


Population of the table using .to_sql function

In [15]:
df_themes.to_sql("themes", engine, if_exists="append", index=False)

641